# 🧠 Fibonacci Reversal XGBoost Trainer (NSE Futures)
This notebook loads historical OHLCV data for NSE Futures stocks, detects potential Fibonacci 0.618 reversal setups, labels outcomes (win/loss), and trains an XGBoost classifier to predict high-confidence signals.

In [ ]:
!pip install -q yfinance ta xgboost scikit-learn pandas numpy matplotlib


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import xgboost as xgb
from ta.momentum import RSIIndicator
from ta.trend import MACD, ADXIndicator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import joblib


In [ ]:
symbols_df = pd.read_csv("futures_list.csv")
symbols = symbols_df["Symbol"].dropna().unique().tolist()
print(f"Loaded {len(symbols)} symbols.")


In [ ]:
def generate_features(df):
    df["RSI"] = RSIIndicator(df["Close"]).rsi()
    macd = MACD(df["Close"])
    df["MACD_Hist"] = macd.macd_diff()
    df["ADX"] = ADXIndicator(df["High"], df["Low"], df["Close"]).adx()
    df["Volume_Change"] = df["Volume"].pct_change() * 100
    return df

def label_reversal_trades(df):
    labels = []
    for i in range(len(df) - 5):
        entry = df.iloc[i]
        future_prices = df.iloc[i+1:i+6]["Close"]
        max_gain = (future_prices.max() - entry["Close"]) / entry["Close"]
        max_loss = (future_prices.min() - entry["Close"]) / entry["Close"]
        if max_gain >= 0.03:
            labels.append(1)  # win
        elif max_loss <= -0.015:
            labels.append(0)  # loss
        else:
            labels.append(None)
    return labels + [None]*5


In [ ]:
data = []
for symbol in symbols:
    try:
        df = yf.download(f"{symbol}.NS", period="1y", interval="1d", progress=False)
        if len(df) < 100:
            continue
        df = generate_features(df)
        high = df['High'].rolling(10).max()
        low = df['Low'].rolling(10).min()
        df['retracement'] = high - (high - low) * 0.618
        df['near_618'] = df['Low'] <= df['retracement']
        df['label'] = label_reversal_trades(df)
        df['Symbol'] = symbol
        data.append(df)
    except Exception as e:
        print(f"Error with {symbol}: {e}")

df_all = pd.concat(data)
df_filtered = df_all[df_all['near_618'] & df_all['label'].notna()]
print("Total labeled samples:", len(df_filtered))


In [ ]:
features = ["RSI", "MACD_Hist", "ADX", "Volume_Change"]
X = df_filtered[features].fillna(0)
y = df_filtered["label"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("🔍 Classification Report:")
print(classification_report(y_test, y_pred))
print("📉 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


In [ ]:
xgb.plot_importance(model)
plt.title("📊 Feature Importance")
plt.show()


In [ ]:
joblib.dump(model, "fibonacci_xgb_model.pkl")
print("✅ Model saved as fibonacci_xgb_model.pkl")
